In [1]:
import igraph as ig
from igraph import Graph

import pickle
import random
import pandas as pd
import numpy as np

from sklearn.metrics import normalized_mutual_info_score
import community as community_louvain
import networkx as nx

from graph_modification import remove_edges, remove_vertices 

#https://igraph.org/python/doc/api/igraph.Graph.html

In [4]:
# Loading serialized datasets
filename = 'datasets.data'

infile = open(filename,'rb')
datasets = pickle.load(infile)
infile.close()

In [5]:
rows = []

pcts = [x*0.1 for x in range(0,10)]
strategies = ['random','betweenness']

for dataset in datasets:
    for strat in strategies:
        for pct in pcts:
            if strat == 'random':
                orders = ['asc']
            else:
                orders = ['asc','desc']

            for order in orders:

                p_size = float(dataset['vertices']['community'].nunique())

                # modify the graph
                if pct == 0:
                    graph = dataset['graph']
                else:
                    graph = remove_edges(dataset['graph'], pct=pct, deep_copy=True, strategy=strat, order=order)

                # louvian
                method = 'Louvain-igraph'

                louvian = ig.Graph.community_multilevel(graph, return_levels=True)
                louvian = louvian[len(louvian)-1]
                p_louvian = len(set(louvian.membership))

                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P',
                             'value':p_louvian})    
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P*/P',
                             'value':p_size/p_louvian})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'Modularity',
                             'value':graph.modularity(louvian.membership)})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'NMI',
                             'value':normalized_mutual_info_score(dataset['vertices']['community'], louvian.membership)})

                # FastGreedy
                method = 'Fastgreedy'

                fg = ig.Graph.community_fastgreedy(graph)
                p_fg = fg.optimal_count
                fg = fg.as_clustering()
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P',
                             'value':p_fg})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P*/P',
                             'value':p_size/p_fg})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'Modularity',
                             'value':graph.modularity(fg.membership)})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'NMI',
                             'value':normalized_mutual_info_score(dataset['vertices']['community'], fg.membership)})
                # Infomap
                method = 'Infomap'

                infomap = ig.Graph.community_infomap(graph)
                p_im = len(set(infomap.membership))
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P',
                             'value':p_im})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P*/P',
                             'value':p_size/p_im})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'Modularity',
                             'value':graph.modularity(infomap.membership)})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'NMI',
                             'value':normalized_mutual_info_score(dataset['vertices']['community'], infomap.membership)})

                # Label Propagation
                method = 'Label Propagation'

                lp = ig.Graph.community_label_propagation(graph)
                p_lp = len(set(lp.membership))
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P',
                             'value':p_lp})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P*/P',
                             'value':p_size/p_lp})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'Modularity',
                             'value':graph.modularity(lp.membership)})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'NMI',
                             'value':normalized_mutual_info_score(dataset['vertices']['community'], lp.membership)})

                # Louvain
                method = 'Louvain'

                # Creating nx Graph to for other Louvain implementation
                nxG = nx.Graph()
                nxG.add_nodes_from([vertex.index for vertex in graph.vs])
                nxG.add_edges_from([edge.tuple for edge in graph.es])

                lv_partition = community_louvain.best_partition(nxG)
                p_lv = len(set(lv_partition.values()))
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P',
                             'value':p_lv})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'P*/P',
                             'value':p_size/p_lv})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'Modularity',
                             'value':graph.modularity(lv_partition.values())})
                rows.append({'dataset':dataset['name'], 'modification':strat+' '+order, 'percentage':pct, 'method':method, 'metric':'NMI',
                             'value':normalized_mutual_info_score(dataset['vertices']['community'], pd.Series(lv_partition.values()))})

experiment_results = pd.DataFrame(rows)

In [6]:
experiment_results.head()

,dataset,modification,percentage,method,metric,value
0,Dancer 01,random asc,0.0,Louvain-igraph,P,9.000000
1,Dancer 01,random asc,0.0,Louvain-igraph,P*/P,0.666667
2,Dancer 01,random asc,0.0,Louvain-igraph,Modularity,0.649290
3,Dancer 01,random asc,0.0,Louvain-igraph,NMI,0.775724
4,Dancer 01,random asc,0.0,Fastgreedy,P,8.000000


In [7]:
experiment_results.to_csv('experiment_results.csv', index=False)